In [1]:
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate

import kagglehub

from constants import dataset_name, opciones, CV, scoring
from utils import cargar_imagenes, evaluar_rendimiento, save_excel_cv

from modelos.RegresionLogistica import regresion_logistica
from modelos.CNN import cnn1, cnn2
from modelos.KNN import knn, knn_with_gridsearch
from modelos.ArbolDeDecision import arbol_decision, arbol_decision_vgg16, arbol_vgg16_pca
from modelos.MobileNetV2 import train_mobilenetv2_model
from modelos.RandomForest import random_forest, rforest_vgg16_pca, rforest_vgg16_pca_hog
from Extractores_con_clasificador.hog_main import main_hog, extract_hog_features
from Extractores_con_clasificador.ann_hog import ann_hog
from Extractores_con_clasificador.svm_hog import svm_hog
from tensorflow.keras.utils import to_categorical
from modelos.MobileNetV2 import train_mobilenetv2_model

import matplotlib.pyplot as plt

# Descargar dataset
path = kagglehub.dataset_download(dataset_name)

# cargar train y test
X_train_rgb, y_train = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/train'), channel_mode="rgb")
X_test_rgb, y_test = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/test'), channel_mode="rgb")
X_train_gray, _ = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/train'), channel_mode="grayscale")
X_test_gray, _ = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/test'), channel_mode="grayscale")
X_train_rgb_64, _ = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/train'), target_size=(64, 64))
X_test_rgb_64, _ = cargar_imagenes(os.path.join(path, 'flowers/flowers/flower_photos/test'), target_size=(64, 64))

# Codificar etiquetas
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

epochs = 100
batch_size = 100
history = []


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## si es regresión logística rgb ##
modelLR_rgb, scoresLR_rgb = regresion_logistica(X_train_rgb, y_train_encoded)
save_excel_cv(scoresLR_rgb, "Regresión Logística RGB")

In [3]:
print(scoresLR_rgb)

{'fit_time': array([265.25171399, 255.92335129, 230.7643075 , 335.09233713,
       241.31467986]), 'score_time': array([159.32398129,   0.78007555,   0.98868942,   0.69361019,
         0.64480305]), 'test_accuracy': array([0.37429379, 0.34887006, 0.32627119, 0.37429379, 0.37711864]), 'test_precision': array([0.360911  , 0.34869617, 0.32552323, 0.36698258, 0.37940236]), 'test_recall': array([0.3612659 , 0.34811868, 0.3235971 , 0.35968507, 0.36655325]), 'test_f1': array([0.35879968, 0.34641024, 0.32419283, 0.35815078, 0.36552693]), 'test_roc_auc': array([0.66949605, 0.66420851, 0.64494095, 0.68413902, 0.68129405])}


In [4]:
## si es regresión logística gray ##
modelLR_gray, scoresLR_gray = regresion_logistica(X_train_gray, y_train_encoded)
save_excel_cv(scoresLR_gray, "Regresión Logística gray")


In [5]:
## si es CNN1 ##
model1 = cnn1()
model1.summary()

history.append(model1.fit(X_train_rgb_64, y_train_encoded,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test_rgb_64, y_test_encoded)))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,877 (93.27 KB)

 Trainable params: 23,877 (93.27 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 117ms/step - accuracy: 0.2378 - loss: 1.5952 - val_accuracy: 0.3200 - val_loss: 1.5676
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.3248 - loss: 1.5003 - val_accuracy: 0.3400 - val_loss: 1.4344
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step - accuracy: 0.3764 - loss: 1.4046 - val_accuracy: 0.4200 - val_loss: 1.4000
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.4146 - loss: 1.3258 - val_accuracy: 0.4400 - val_loss: 1.2790
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.4341 - loss: 1.2942 - val_accuracy: 0.4600 - val_loss: 1.2457
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step - accuracy: 0.4544 - loss: 1.2435 - val_accuracy: 0.4600 - val_loss: 1.2347
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 0.4742 - loss: 1.2359 - val_accuracy: 0.5000 - val_loss: 1.2060
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 104ms/step - accuracy: 0.4787 - loss: 1.2377 - val_accu

In [6]:
## si es CNN2 ##    
model2 = cnn2()
model2.summary()

history.append(model2.fit(X_train_rgb_64, y_train_encoded,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(X_test_rgb_64, y_test_encoded)))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,437 (275.14 KB)

 Trainable params: 70,245 (274.39 KB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 25s 531ms/step - accuracy: 0.4210 - loss: 1.4324 - val_accuracy: 0.2000 - val_loss: 1.7579
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 528ms/step - accuracy: 0.5943 - loss: 1.0571 - val_accuracy: 0.3600 - val_loss: 1.4836
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 526ms/step - accuracy: 0.6169 - loss: 0.9883 - val_accuracy: 0.2800 - val_loss: 1.4910
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 527ms/step - accuracy: 0.6574 - loss: 0.9008 - val_accuracy: 0.2600 - val_loss: 1.6217
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 522ms/step - accuracy: 0.6649 - loss: 0.8755 - val_accuracy: 0.2200 - val_loss: 1.6345
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 525ms/step - accuracy: 0.6883 - loss: 0.8125 - val_accuracy: 0.2600 - val_loss: 1.6314
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 527ms/step - accuracy: 0.6948 - loss: 0.8044 - val_accuracy: 0.2600 - val_loss: 1.6860
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 524ms/step - accuracy: 0.7185 - loss: 0.7533 - 

In [ ]:
## KNN ## 
_, _, scoresKNN = knn(X_train_rgb, y_train_encoded, X_test_rgb)

save_excel_cv(scoresKNN, "KNN")

In [ ]:
modelKNN, pcaKNN, scoresKNN = knn_with_gridsearch(X_train_rgb_64, y_train_encoded, X_test_rgb_64, y_test_encoded)

Mejores parámetros: {'knn__n_neighbors': 7, 'pca__n_components': 30}
   param_pca__n_components  param_knn__n_neighbors  mean_test_accuracy  \
2                       30                       7            0.435311   
3                       50                       7            0.430226   
0                       30                       3            0.413842   
1                       50                       3            0.412429   

   mean_test_precision  mean_test_recall  mean_test_f1  mean_test_roc_auc  
2             0.436485          0.421142      0.416222           0.728680  
3             0.440036          0.412501      0.406571           0.725369  
0             0.425091          0.403926      0.402425           0.687863  
1             0.424728          0.400201      0.397598           0.686073  


In [ ]:
model_tree = arbol_decision(X_train_rgb_64, y_train_encoded)
evaluar_rendimiento(model_tree, X_test_rgb_64, y_test, "Árbol de Decisión")

KeyboardInterrupt: 

In [ ]:
model_tree = arbol_decision_vgg16(X_train_rgb_64, y_train_encoded)
evaluar_rendimiento(model_tree, X_test_rgb_64, y_test, "Árbol de Decisión VGG16")

In [ ]:
model_tree = arbol_vgg16_pca(X_train_rgb_64, y_train_encoded)
# Evaluar el modelo con las funciones definidas previamente
evaluar_rendimiento(model_tree, X_test_rgb_64, y_test, "Árbol de Decisión VGG16 PCA")

In [ ]:
num_classes = len(np.unique(y_train_encoded))  # Asegurar que tenemos el número correcto de clases
y_train_onehot = to_categorical(y_train_encoded, num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes)

# Llamada a la función de entrenamiento
model_1 = train_mobilenetv2_model(X_train_rgb, y_train_onehot, X_test_rgb, y_test_onehot, epochs=10, batch_size=32, dense_units=1024, dropout_rate=0.5, learning_rate=0.001)
model_2 = train_mobilenetv2_model(X_train_rgb_64, y_train_onehot, X_test_rgb_64, y_test_onehot, input_shape=(64, 64, 3), epochs=10, batch_size=32, dense_units=1024, dropout_rate=0.5, learning_rate=0.001)
model_3 = train_mobilenetv2_model(X_train_rgb_64, y_train_onehot, X_test_rgb_64, y_test_onehot, input_shape=(64, 64, 3),epochs=10, batch_size=32, dense_units=1024, dropout_rate=0.5, learning_rate=0.0001)
model_4 = train_mobilenetv2_model(X_train_rgb_64, y_train_onehot, X_test_rgb_64, y_test_onehot, input_shape=(64, 64, 3),epochs=10, batch_size=32, dense_units=1024, dropout_rate=0.3, learning_rate=0.001)

d:\Documentos\Master Informatica Industrial y Robotica\VA I\Trabajo_git\clasificacion-supervisada\modelos\MobileNetV2.py:13: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)


Epoch 1/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 95s 771ms/step - accuracy: 0.7031 - loss: 0.8184 - val_accuracy: 0.8000 - val_loss: 0.7273
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 83s 747ms/step - accuracy: 0.9016 - loss: 0.2687 - val_accuracy: 0.7600 - val_loss: 0.7518
Epoch 3/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 79s 712ms/step - accuracy: 0.9156 - loss: 0.2395 - val_accuracy: 0.8200 - val_loss: 0.7202
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 77s 696ms/step - accuracy: 0.9380 - loss: 0.1680 - val_accuracy: 0.7800 - val_loss: 0.8725
Epoch 5/10
 79/111 ━━━━━━━━━━━━━━━━━━━━ 22s 714ms/step - accuracy: 0.9434 - loss: 0.1593

KeyboardInterrupt: 

In [ ]:
modelRF, scoresRF = random_forest(X_train_rgb_64, y_train, X_test_rgb_64)
save_excel_cv(scoresRF, "RANDOM FOREST")

In [ ]:
rf_model, scoresRFVGGPCA = rforest_vgg16_pca(X_train_rgb, y_train, X_test_rgb)        
save_excel_cv(scoresRF, "RANDOM FOREST VGG16 PCA")

111/111 ━━━━━━━━━━━━━━━━━━━━ 687s 6s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step


In [ ]:
rf_model, scoresRFHOG = rforest_vgg16_pca_hog(X_train_rgb_64, y_train, X_test_rgb_64)
save_excel_cv(scoresRFHOG, "RANDOM FOREST VGG16 PCA")

In [ ]:
from skimage.feature import hog
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from Extractores_con_clasificador.ann_hog import ann_hog
from Extractores_con_clasificador.svm_hog import svm_hog
from sklearn.svm import SVC

def ann_hog(X_train_hog, y_train, X_test_hog, y_test, num_classes):
    # Convertir las etiquetas a one-hot encoding
    y_train_onehot = to_categorical(y_train, num_classes)
    y_test_onehot = to_categorical(y_test, num_classes)

    # Crear el modelo ANN
    ann_model = Sequential([
        Dense(512, activation='relu', input_dim=X_train_hog.shape[1]),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')  # num_classes = cantidad de clases
    ])

    # Compilar el modelo
    ann_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo ANN
    ann_model.fit(X_train_hog, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_hog, y_test_onehot))

    # Obtener predicciones
    y_pred_ann = ann_model.predict(X_test_hog)
    y_pred_ann_classes = np.argmax(y_pred_ann, axis=1)

    # Calcular precisión
    accuracy_ann = accuracy_score(y_test, y_pred_ann_classes)
    print(f"Precisión ANN con HOG: {accuracy_ann:.2f}")

    # Generar reporte de clasificación
    print("\nReporte de ANN con HOG:")
    print(classification_report(y_test, y_pred_ann_classes))

    return ann_model

def svm_hog(X_train_hog, y_train, X_test_hog, y_test):
    # Crear y entrenar el clasificador SVM
    svm_model = SVC(kernel="linear", random_state=42)
    svm_model.fit(X_train_hog, y_train)

    # Hacer predicciones en el conjunto de prueba
    y_pred_svm = svm_model.predict(X_test_hog)

    # Evaluar precisión
    accuracy_svm = accuracy_score(y_test, y_pred_svm)
    print(f"Precisión SVM con HOG: {accuracy_svm:.2f}")

    # Generar reporte de clasificación
    print("\nReporte de SVM con HOG:")
    print(classification_report(y_test, y_pred_svm))

    return svm_model

def extract_hog_features(images):
    hog_features = []
    for img in images:
        feature = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(feature)
    return np.array(hog_features)

def main_hog(X_train, X_test, y_train, y_test, num_classes):
    # Extraer características HOG
    X_train_hog = extract_hog_features(X_train)
    X_test_hog = extract_hog_features(X_test)

    # Entrenar y evaluar ANN con características HOG
    print("Entrenando y evaluando ANN con características HOG...")
    ann_model = ann_hog(X_train_hog, y_train, X_test_hog, y_test, num_classes)

    # Entrenar y evaluar SVM con características HOG
    print("\nEntrenando y evaluando SVM con características HOG...")
    svm_model = svm_hog(X_train_hog, y_train, X_test_hog, y_test)

    return ann_model, svm_model

In [ ]:
print(type(hog))  # Debe imprimir: <class 'function'>


<class 'function'>


In [ ]:
num_classes = len(np.unique(y_train))  # Suponiendo que y_train contiene las etiquetas
main_hog(X_train_gray, X_test_gray, y_train_encoded, y_test_encoded, num_classes)

Entrenando y evaluando ANN con características HOG...


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.2620 - loss: 2.2188 - val_accuracy: 0.3400 - val_loss: 1.5087
Epoch 2/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.3629 - loss: 1.4627 - val_accuracy: 0.3200 - val_loss: 1.5507
Epoch 3/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.3891 - loss: 1.4153 - val_accuracy: 0.2600 - val_loss: 1.5111
Epoch 4/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.4106 - loss: 1.3725 - val_accuracy: 0.3600 - val_loss: 1.4657
Epoch 5/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.4087 - loss: 1.3856 - val_accuracy: 0.3400 - val_loss: 1.4608
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Precisión ANN con HOG: 0.34

Reporte de ANN con HOG:
              precision    recall  f1-score   support

           0       0.67      0.20      0.31        10
           1       0.33      0.40      0.36        10
           2       0.31      0.40      0.35        10
           3       0.33      0.10      0.15        1

(<Sequential name=sequential, built=True>,
 SVC(kernel='linear', random_state=42))

In [ ]:
## diagrama de cajas validación cru
data = [scoresLR_rgb['test_accuracy'], scoresLR_gray['test_accuracy'], scoresKNN['test_accuracy']]
_, ax = plt.subplots()
ax.set_title('Modelos')
ax.boxplot(data,labels=['LR_RGB','LR_GRAY','KNN'])

NameError: name 'scoresLR_rgb' is not defined